In [23]:
import pandas as pd
movies=pd.read_csv('/movies.csv')
ratings=pd.read_csv('/ratings.csv')

In [24]:
movies["year"]=movies["title"].str.extract(r'\((\d{4})\)')
genres=movies["genres"]
new_genres=[]
for genre in genres:
    new_genres.append(genre.split("|"))
unique_genres=set()
for genre_list in new_genres:
    for genre in genre_list:
        unique_genres.add(genre)
unique_genres.remove('(no genres listed)')

In [25]:
merged_df = pd.merge(movies, ratings, on='movieId')

In [28]:
def assign_rating(row):
    if row['rating'] < 3:
        return [1, 0, 0]
    elif row['rating'] == 3:
        return [0, 1, 0]
    else:
        return [0, 0, 1]

merged_df[['rating_low', 'rating_medium', 'rating_high']] = merged_df.apply(assign_rating, axis=1, result_type='expand')

def assign_year(row):
    try:
        if int(row["year"])<2000:
            return [1,0]
        else:
            return [0,1]
    except:
        return [0,0]

merged_df[["year_old","year_new"]] = merged_df.apply(assign_year,axis=1, result_type='expand')

genres_df = merged_df['genres'].str.get_dummies(sep='|')
merged_df = pd.concat([merged_df, genres_df], axis=1)
merged_df

,movieId,title,genres,year,userId,rating,timestamp,rating_low,rating_medium,rating_high,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,1,4.0,964982703,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,5,4.0,847434962,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,7,4.5,1106635946,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,15,2.5,1510577970,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,17,4.5,1305696483,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017,184,4.0,1537109082,0,0,1,...,0,0,0,0,0,0,0,0,0,0
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017,184,3.5,1537109545,0,0,1,...,0,0,0,0,0,0,0,0,0,0
100833,193585,Flint (2017),Drama,2017,184,3.5,1537109805,0,0,1,...,0,0,0,0,0,0,0,0,0,0
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018,184,3.5,1537110021,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Dropping unwanted collumns

In [37]:
merged_df=merged_df.drop(columns=['movieId','title','genres', 'year', 'userId', 'rating', 'timestamp', '(no genres listed)'])

In [40]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(merged_df, min_support=0.02, use_colnames=True)

association_rules(frequent_itemsets, 
                  metric='confidence', 
                  min_threshold=0.2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(rating_low),(year_old),0.189149,0.610546,0.109802,0.580506,0.950799,-0.005682,0.928391
1,(year_new),(rating_low),0.389276,0.189149,0.079317,0.203755,1.077222,0.005686,1.018344
2,(rating_low),(year_new),0.189149,0.389276,0.079317,0.419336,1.077222,0.005686,1.051769
3,(rating_low),(Action),0.189149,0.303810,0.062031,0.327951,1.079459,0.004566,1.035921
4,(Action),(rating_low),0.303810,0.189149,0.062031,0.204178,1.079459,0.004566,1.018886
...,...,...,...,...,...,...,...,...,...
1060,"(Comedy, Animation)","(Children, Adventure)",0.037923,0.050716,0.023146,0.610356,12.034772,0.021223,2.436283
1061,"(Comedy, Adventure)","(Children, Animation)",0.076094,0.052947,0.023146,0.304184,5.745017,0.019118,1.361067
1062,"(Animation, Adventure)","(Children, Comedy)",0.040938,0.051797,0.023146,0.565407,10.915830,0.021026,2.181818
1063,(Children),"(Comedy, Animation, Adventure)",0.091317,0.025656,0.023146,0.253475,9.879949,0.020804,1.305174
